<a href="https://colab.research.google.com/github/hitesha14/Rising2021_Timeseries/blob/main/Copy_of_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Dropout
tf.random.set_seed(123)
np.random.seed(123)

In [ ]:
df = pd.read_csv(r'https://api.covid19india.org/csv/latest/case_time_series.csv')

In [ ]:
df.head()

,Date,Date_YMD,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,30 January 2020,2020-01-30,1,1,0,0,0,0
1,31 January 2020,2020-01-31,0,1,0,0,0,0
2,1 February 2020,2020-02-01,0,1,0,0,0,0
3,2 February 2020,2020-02-02,1,2,0,0,0,0
4,3 February 2020,2020-02-03,1,3,0,0,0,0


In [ ]:
df.describe()

,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
count,478.000000,4.780000e+02,478.000000,4.780000e+02,478.000000,478.000000
mean,54995.529289,6.294503e+06,48249.930962,5.657688e+06,617.089958,87415.654812
std,87250.607082,6.199014e+06,77991.369440,5.565253e+06,895.591405,75238.905902
min,0.000000,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,7371.250000,1.673925e+05,3812.250000,7.382975e+04,100.000000,4777.250000
50%,23662.000000,5.858972e+06,20573.000000,4.799650e+06,342.000000,92279.500000
75%,61675.500000,1.063689e+07,58095.750000,1.029560e+07,775.000000,152591.750000
max,414280.000000,2.628786e+07,422391.000000,2.306347e+07,4529.000000,294969.000000


In [ ]:
df.describe()

,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
count,478.000000,4.780000e+02,478.000000,4.780000e+02,478.000000,478.000000
mean,54995.529289,6.294503e+06,48249.930962,5.657688e+06,617.089958,87415.654812
std,87250.607082,6.199014e+06,77991.369440,5.565253e+06,895.591405,75238.905902
min,0.000000,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,7371.250000,1.673925e+05,3812.250000,7.382975e+04,100.000000,4777.250000
50%,23662.000000,5.858972e+06,20573.000000,4.799650e+06,342.000000,92279.500000
75%,61675.500000,1.063689e+07,58095.750000,1.029560e+07,775.000000,152591.750000
max,414280.000000,2.628786e+07,422391.000000,2.306347e+07,4529.000000,294969.000000


In [ ]:
len(df.index)

478

df.tail() This function returns last n rows from the object based on position.

In [ ]:
df.tail(10)

,Date,Date_YMD,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
468,12 May 2021,2021-05-12,362632,23703007,352005,19728566,4128,257762
469,13 May 2021,2021-05-13,343005,24046012,344676,20073242,4000,261762
470,14 May 2021,2021-05-14,326256,24372268,353200,20426442,3889,265651
471,15 May 2021,2021-05-15,310757,24683025,362547,20788989,4077,269728
472,16 May 2021,2021-05-16,281837,24964862,378526,21167515,4098,273826
473,17 May 2021,2021-05-17,263021,25227883,422391,21589906,4334,278160
474,18 May 2021,2021-05-18,267246,25495129,389758,21979664,4529,282689
475,19 May 2021,2021-05-19,276193,25771322,369005,22348669,3877,286566
476,20 May 2021,2021-05-20,259242,26030564,357173,22705842,4209,290775
477,21 May 2021,2021-05-21,257299,26287863,357625,23063467,4194,294969


In [ ]:
validate = df['Daily Confirmed'].tail(10)
df.drop(df['Daily Confirmed'].tail(10).index,inplace=True)
df['Date_YMD'] =  pd.to_datetime(df['Date_YMD'])

In [ ]:
uni_data = df['Daily Confirmed']
uni_data.index = df['Date_YMD']
uni_data.head()

Date_YMD
2020-01-30    1
2020-01-31    0
2020-02-01    0
2020-02-02    1
2020-02-03    1
Name: Daily Confirmed, dtype: int64

In [ ]:
uni_data = uni_data.values
scaler_x = preprocessing.MinMaxScaler()
x_rescaled = scaler_x.fit_transform(uni_data.reshape(-1, 1))

In [ ]:
def custom_ts_univariate_data_prep(dataset, start, end, window, horizon):
  X = []
  y = []

  start = start + window
  if end is None:
    end = len(dataset) - horizon

  for i in range(start, end):
    indicesx = range(i-window, i)
    X.append(np.reshape(dataset[indicesx], (window, 1)))
    indicesy = range(i,i+horizon)
    y.append(dataset[indicesy])
  return np.array(X), np.array(y)

In [ ]:
univar_hist_window = 10
horizon = 10
TRAIN_SPLIT = 300
x_train_uni, y_train_uni = custom_ts_univariate_data_prep(x_rescaled, 0, TRAIN_SPLIT,univar_hist_window, horizon)
x_val_uni, y_val_uni = custom_ts_univariate_data_prep(x_rescaled, TRAIN_SPLIT, None,univar_hist_window,horizon)

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target horizon')
print (y_train_uni[0])

Single window of past history
[[2.4138264e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 [2.4138264e-06]
 [2.4138264e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]]

 Target horizon
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
BATCH_SIZE = 25
BUFFER_SIZE = 150

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train_uni.shape[1], x_train_uni.shape[2])))
model.add(MaxPool1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(horizon))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model_path = r'CNN_Univarient_1.h5'

In [ ]:
EVALUATION_INTERVAL = 50
EPOCHS = 150
history = model.fit(train_univariate, epochs=EPOCHS,steps_per_epoch=EVALUATION_INTERVAL,validation_data=val_univariate, validation_steps=50,verbose =1,
                              callbacks =[tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min'),tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)])

Epoch 1/150
50/50 [==============================] - 1s 9ms/step - loss: 0.0055 - val_loss: 0.0295
Epoch 2/150
50/50 [==============================] - 0s 3ms/step - loss: 0.0016 - val_loss: 0.0237
Epoch 3/150
50/50 [==============================] - 0s 3ms/step - loss: 0.0013 - val_loss: 0.0264
Epoch 4/150
50/50 [==============================] - 0s 3ms/step - loss: 0.0012 - val_loss: 0.0240
Epoch 5/150
50/50 [==============================] - 0s 3ms/step - loss: 9.5409e-04 - val_loss: 0.0245
Epoch 6/150
50/50 [==============================] - 0s 3ms/step - loss: 8.6650e-04 - val_loss: 0.0269
Epoch 7/150
50/50 [==============================] - 0s 3ms/step - loss: 7.7976e-04 - val_loss: 0.0232
Epoch 8/150
50/50 [==============================] - 0s 3ms/step - loss: 7.3406e-04 - val_loss: 0.0210
Epoch 9/150
50/50 [==============================] - 0s 3ms/step - loss: 7.0732e-04 - val_loss: 0.0219
Epoch 10/150
50/50 [==============================] - 0s 3ms/step - loss: 6.9962e-04 - va

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
Trained_model = tf.keras.models.load_model(model_path)

In [ ]:
# Show the model architecture
Trained_model.summary()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'], loc='upper left')
plt.rcParams["figure.figsize"] = [16,9]
plt.show()

In [ ]:
uni = df['Daily Confirmed']
validatehori = uni.tail(10)

In [ ]:
validatehist = validatehori.values
scaler_val = preprocessing.MinMaxScaler()
val_rescaled = scaler_x.fit_transform(validatehist.reshape(-1, 1))

In [ ]:
val_rescaled = val_rescaled.reshape((1, val_rescaled.shape[0], 1))

In [ ]:
Predicted_results = Trained_model.predict(val_rescaled)

In [ ]:
Predicted_results

In [ ]:
Predicted_inver_res = scaler_x.inverse_transform(Predicted_results)

In [ ]:
Predicted_inver_res

In [ ]:
from sklearn import metrics
def timeseries_evaluation_metrics_func(y_true, y_pred):
    
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

In [ ]:
timeseries_evaluation_metrics_func(validate,Predicted_inver_res[0])

In [ ]:
plt.plot( list(validate))
plt.plot( list(Predicted_inver_res[0]))
plt.title("Actual vs Predicted")
plt.ylabel("Traffic volume")
plt.legend(('Actual','predicted'))
plt.show()